# Assignment 03 - Mining Frequent Itemsets using FP-Growth Algorithm
## 1. Preparation
### 1.1 Requirements

1. Operating System : Linux Mint 18.3 Sylvia
2. Apache Spark 2.4.0 Binary (https://spark.apache.org/downloads.html)
3. PySpark 2.4.0 (Apache Spark Python API)
4. Findspark 1.3.0 (Python's library)
5. Jupyter Notebook (https://jupyter.org/install)

### 1.2 Description of Dataset
* Dataset's name : [Brazilian E-Commerce Public Dataset by Olist](https://www.kaggle.com/olistbr/brazilian-ecommerce)
* Description : a Brazilian ecommerce public dataset of orders made at Olist Store. The dataset has information of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil. Its features allows viewing an order from multiple dimensions: from order status, price, payment and freight performance to customer location, product attributes and finally reviews written by customers. 
* There are nine data sources in this dataset, but I'll use only two data sources that required to mining frequent itemsets, that is **olist_order_items_dataset.csv** and **olist_products_dataset.csv**
    
| Data Source | Number of Rows | Number of Columns |
|---|---|---|
| olist_customers_dataset.csv | 99.4k | 5 |
| olist_geolocation_dataset.csv | 1.00m | 5 |
| olist_order_items_dataset.csv | 113k | 7 |
| olist_order_payments_dataset.csv | 104k | 5 |
| olist_order_reviews_dataset.csv | 100.0k | 7 |
| olist_orders_dataset.csv | 99.4k | 8 |
| olist_products_dataset.csv | 33.0k | 9 |
| olist_sellers_dataset.csv | 3095 | 4 |
| product_category_name_translation.csv | 71 | 2 |

![data scheme](img/olistdata.png)

## 2. Spark Initialization

In [1]:
# Import findspark to make pyspark importable as a regular library
import findspark
findspark.init('/home/mocatfrio/spark') 

# /home/mocatfrio/spark has symbolic link to /bin/spark-2.4.0-bin-hadoop2.7

In [2]:
# Import SparkSession
from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession \
    .builder \
    .appName("Frequent Itemsets") \
    .getOrCreate()

In [3]:
# Print spark object ID
print(spark)

## 3. Load Dataset
### 3.1 Order items Dataset

In [4]:
# Load the dataset
df_order = spark.read.load("/home/mocatfrio/Documents/projects/big-data/datasets/brazilian_ecommerce/olist_order_items_dataset.csv", \
                     format="csv", sep=",", inferSchema="true", header="true")

In [5]:
# Print top 20 rows data
df_order.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [6]:
# Count data rows
df_order.count()

112650

In [7]:
# inferSchema is used to inference the actual datatype of columns, especially for dates and timestamp
df_order.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



In [8]:
# Register the dataframe as a SQL temporary view
df_order.createOrReplaceTempView("order")

### 3.1 Products Dataset

In [9]:
# Load the dataset
df_product = spark.read.load("/home/mocatfrio/Documents/projects/big-data/datasets/brazilian_ecommerce/olist_products_dataset.csv", \
                     format="csv", sep=",", inferSchema="true", header="true")

In [10]:
df_product.show()

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [11]:
df_product.count()

32951

In [12]:
df_product.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)



In [13]:
df_product.createOrReplaceTempView("product")

## 4. Frequent Pattern Mining using FP-Growth Algorithm
Apache Spark has provides a library to mining frequent itemsets using FP-Growth algorithm. First step of FP-growth is to calculate item frequencies and identify frequent items. Then, FP-Growth uses a suffix tree (FP-Tree) structure to encode transactions without generating candidate sets explicitly, which are usually expensive to generate. After that, the frequent itemsets can be extracted from the FP-Tree. For further explanation about FP-Growth, please visit http://hareenlaks.blogspot.com/2011/06/fp-tree-example-how-to-identify.html


### 4.1 Preprocess Data
First, we need to preprocess the original data because FP-Growth algorithm from Spark can only process 2-column data. i.e. id and items, for example:

| id|       items|
|---|------------|
|  0|   [1, 2, 5]|
|  1|[1, 2, 3, 5]|
|  2|      [1, 2]|

In [14]:
# Make new id for each products named "item_id" using row_number()
df_prep1 = spark.sql("SELECT `product_id`, ROW_NUMBER() OVER (ORDER BY `product_id`) AS `item_id` \
                    FROM product")
df_prep1.show(truncate=False)

+--------------------------------+-------+
|product_id                      |item_id|
+--------------------------------+-------+
|00066f42aeeb9f3007548bb9d3f33c38|1      |
|00088930e925c41fd95ebfe695fd2655|2      |
|0009406fd7479715e4bef61dd91f2462|3      |
|000b8f95fcb9e0096488278317764d19|4      |
|000d9be29b5207b54e86aa1b1ac54872|5      |
|0011c512eb256aa0dbbb544d8dffcf6e|6      |
|00126f27c813603687e6ce486d909d01|7      |
|001795ec6f1b187d37335e1c4704762e|8      |
|001b237c0e9bb435f2e54071129237e9|9      |
|001b72dfd63e9833e8c02742adf472e3|10     |
|001c5d71ac6ad696d22315953758fa04|11     |
|00210e41887c2a8ef9f791ebc780cc36|12     |
|002159fe700ed3521f46cfcf6e941c76|13     |
|0021a87d4997a48b6cef1665602be0f5|14     |
|00250175f79f584c14ab5cecd80553cd|15     |
|002552c0663708129c0019cc97552d7d|16     |
|002959d7a0b0990fe2d69988affcbc80|17     |
|002af88741ba70c7b5cf4e4a0ad7ef85|18     |
|002c6dab60557c48cfd6c2222ef7fd76|19     |
|002d4ea7c04739c130bb74d7e7cd1694|20     |
+----------

In [15]:
# Register the dataframe as a SQL temporary view
df_prep1.createOrReplaceTempView("product_prep")

In [16]:
# Make new id for each order named "id" using row_number()
df_prep2 = spark.sql("SELECT `order_id`, ROW_NUMBER() OVER (ORDER BY `order_id`) AS `id`, COUNT(`order_id`) AS `num_of_items` \
                    FROM order \
                    GROUP BY `order_id`")

df_prep2.show(truncate=False)

+--------------------------------+---+------------+
|order_id                        |id |num_of_items|
+--------------------------------+---+------------+
|00010242fe8c5a6d1ba2dd792cb16214|1  |1           |
|00018f77f2f0320c557190d7a144bdd3|2  |1           |
|000229ec398224ef6ca0657da4fc703e|3  |1           |
|00024acbcdf0a6daa1e931b038114c75|4  |1           |
|00042b26cf59d7ce69dfabb4e55b4fd9|5  |1           |
|00048cc3ae777c65dbb7d2a0634bc1ea|6  |1           |
|00054e8431b9d7675808bcb819fb4a32|7  |1           |
|000576fe39319847cbb9d288c5617fa6|8  |1           |
|0005a1a1728c9d785b8e2b08b904576c|9  |1           |
|0005f50442cb953dcd1d21e1fb923495|10 |1           |
|00061f2a7bc09da83e415a52dc8a4af1|11 |1           |
|00063b381e2406b52ad429470734ebd5|12 |1           |
|0006ec9db01a64e59a68b2c340bf65a7|13 |1           |
|0008288aa423d2a3f00fcb17cd7d8719|14 |2           |
|0009792311464db532ff765bf7b182ae|15 |1           |
|0009c9a17f916a706d71784483a5d643|16 |1           |
|000aed2e25d

In [17]:
# Register the dataframe as a SQL temporary view
df_prep2.createOrReplaceTempView("order_prep")

In [18]:
# Join the order and products dataframe by product_id, so its product_id converted to item_id 
# Then, group by "id" and "item_id" column, so the "items" will contain unique item_id
df_prep3 = spark.sql("SELECT op.id, p.item_id \
                    FROM order AS o \
                    JOIN product_prep AS p \
                        ON o.product_id = p.product_id \
                    JOIN order_prep AS op \
                        ON o.order_id = op.order_id \
                    GROUP BY op.id, p.item_id \
                    ORDER BY op.id ASC")

In [19]:
# Check the datatype
df_prep3.printSchema()

root
 |-- id: integer (nullable = true)
 |-- item_id: integer (nullable = true)



In [20]:
# Import StringType
from pyspark.sql.types import StringType

# Convert the datatypes from int to string
df_prep3 = df_prep3.withColumn('id', df_prep3.id.cast(StringType())).withColumn('item_id', df_prep3.item_id.cast(StringType()))
    
df_prep3.show()

+---+-------+
| id|item_id|
+---+-------+
|  1|   8629|
|  2|  29598|
|  3|  25668|
|  4|  15323|
|  5|  22080|
|  6|  30848|
|  7|  18182|
|  8|  11123|
|  9|   6385|
| 10|   9013|
| 11|  27628|
| 12|  31102|
| 13|  19743|
| 14|   7080|
| 15|  18083|
| 16|   8238|
| 17|  10354|
| 18|  23217|
| 19|  12287|
| 20|  11081|
+---+-------+
only showing top 20 rows



In [21]:
# After converting the datatype
df_prep3.printSchema()

root
 |-- id: string (nullable = true)
 |-- item_id: string (nullable = true)



In [22]:
# Import functions sql
from pyspark.sql import functions as F

# Group by "id" and merge/combine the "item_id" column to array in "items" column
df_prep = df_prep3.groupBy("id").agg(F.collect_list(F.col("item_id")).alias("items"))

df_prep.show(100)

+-----+--------------+
|   id|         items|
+-----+--------------+
|10096|       [21884]|
|10351|        [9274]|
|10436|       [15473]|
| 1090|        [8965]|
|11078|       [22549]|
|11332|       [12206]|
|11563|       [24173]|
| 1159|       [14100]|
|11722|       [14509]|
|11888|        [6985]|
|12394|        [8685]|
|12529|       [17667]|
|12847|       [28719]|
|13192|       [10894]|
|13282|       [15155]|
|13442|        [2798]|
|13610|       [18717]|
|13772|       [16240]|
|13865|       [10623]|
|14157|       [17176]|
|14204|       [14751]|
| 1436|       [23754]|
|14369|       [11169]|
|14838|       [14200]|
|14887|       [28550]|
|14899|       [32266]|
| 1512|       [17303]|
|15269|        [2917]|
|15271|       [19554]|
|15555|       [13485]|
|15574|        [8614]|
|15634|       [24356]|
| 1572|       [32073]|
|16250|       [27001]|
|16320|        [4076]|
|16504|        [6682]|
|16576|[20538, 25213]|
|16974|        [2715]|
|17401|        [3289]|
|17427|       [26832]|
|17506|    

In [23]:
df_prep.count()

98666

### 4.2 FP-Growth Algorithm
Make at least 3 models with different minimum support and minimum confidence values.

In [24]:
# Import FPGrowth 
from pyspark.ml.fpm import FPGrowth

In [25]:
# 1. Initial testing
fpGrowth1 = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model1 = fpGrowth1.fit(df_prep)

In [26]:
# Display frequent itemsets.
model1.freqItemsets.show()

+-----+----+
|items|freq|
+-----+----+
+-----+----+



In [27]:
# 2. Model 1 doesn't display any results, so I decrease the min support and min confidence
fpGrowth2 = FPGrowth(itemsCol="items", minSupport=0.1, minConfidence=0.2)
model2 = fpGrowth2.fit(df_prep)

In [28]:
# Display frequent itemsets.
model2.freqItemsets.show()

+-----+----+
|items|freq|
+-----+----+
+-----+----+



In [29]:
# 3. Model 2 doesn't display any results, so I decrease the min support and min confidence
fpGrowth3 = FPGrowth(itemsCol="items", minSupport=0.0005, minConfidence=0.0005)
model3 = fpGrowth3.fit(df_prep)

In [30]:
# Display frequent itemsets.
model3.freqItemsets.show()

+-------+----+
|  items|freq|
+-------+----+
|[19743]| 467|
|[22113]| 431|
| [8614]| 352|
|[27040]| 323|
| [7365]| 311|
|[10868]| 306|
| [7080]| 291|
|[10841]| 287|
| [2795]| 269|
| [5693]| 259|
| [8052]| 255|
|[16087]| 225|
|[11747]| 194|
|[28971]| 194|
|[24087]| 187|
|[21303]| 172|
| [8685]| 160|
|[21672]| 158|
|[29514]| 156|
| [6980]| 156|
+-------+----+
only showing top 20 rows



In [31]:
# Display generated association rules.
model3.associationRules.show()

+----------+----------+----------+----+
|antecedent|consequent|confidence|lift|
+----------+----------+----------+----+
+----------+----------+----------+----+



In [32]:
# 4. Model 3 display results but doesn't display generated association rules
fpGrowth4 = FPGrowth(itemsCol="items", minSupport=0.00005, minConfidence=0.00005)
model4 = fpGrowth4.fit(df_prep)

In [33]:
# Display frequent itemsets.
model4.freqItemsets.show()

+-------+----+
|  items|freq|
+-------+----+
|[19907]|  42|
|[17443]|   5|
|[25857]|  26|
| [7413]|   8|
| [1420]|  20|
|[16023]|   8|
| [2247]|   7|
|[24026]|   5|
|[16695]|   5|
| [2966]|  16|
|[15686]|  14|
|[31869]|  12|
|[30822]|   7|
| [5242]|   6|
| [3804]|   6|
|[18514]|  11|
|[19409]|  10|
| [7728]|   6|
| [2915]|   9|
|[12862]|   5|
+-------+----+
only showing top 20 rows



In [34]:
# Display generated association rules.
model4.associationRules.show()

+----------+----------+--------------------+------------------+
|antecedent|consequent|          confidence|              lift|
+----------+----------+--------------------+------------------+
|   [29569]|   [11885]| 0.17647058823529413| 829.1260504201681|
|    [8228]|    [7132]| 0.14814814814814814|131.68635301968632|
|    [3200]|   [28328]|  0.4117647058823529| 625.0334841628959|
|   [19743]|    [6980]| 0.06209850107066381| 39.27570965793664|
|   [19743]|   [31285]| 0.01284796573875803|45.273478127867854|
|   [12405]|   [12115]|  0.3157894736842105|1947.3552631578946|
|    [8614]|    [7365]|             0.03125|  9.91418810289389|
|    [8614]|   [10841]|0.019886363636363636| 6.836613082039912|
|    [8614]|    [1518]|0.017045454545454544| 16.81806818181818|
|   [10377]|   [31536]| 0.19101123595505617| 336.5413322632424|
|    [6980]|   [19743]|  0.1858974358974359| 39.27570965793664|
|    [7365]|    [8614]| 0.03536977491961415|  9.91418810289389|
|    [7365]|    [7080]| 0.01607717041800

In [35]:
# Transform examines the input items against all the association rules and summarize the
# Consequents as prediction
model4.transform(df_prep).show(50, truncate=False)

+-----+--------------+-------------------+
|id   |items         |prediction         |
+-----+--------------+-------------------+
|10096|[21884]       |[]                 |
|10351|[9274]        |[]                 |
|10436|[15473]       |[]                 |
|1090 |[8965]        |[]                 |
|11078|[22549]       |[]                 |
|11332|[12206]       |[]                 |
|11563|[24173]       |[]                 |
|1159 |[14100]       |[]                 |
|11722|[14509]       |[]                 |
|11888|[6985]        |[]                 |
|12394|[8685]        |[]                 |
|12529|[17667]       |[]                 |
|12847|[28719]       |[]                 |
|13192|[10894]       |[]                 |
|13282|[15155]       |[]                 |
|13442|[2798]        |[]                 |
|13610|[18717]       |[]                 |
|13772|[16240]       |[]                 |
|13865|[10623]       |[]                 |
|14157|[17176]       |[]                 |
|14204|[147

In [36]:
# 5. I'll try once more by decreasing min support and min confidence more
fpGrowth5 = FPGrowth(itemsCol="items", minSupport=0.000005, minConfidence=0.000005)
model5 = fpGrowth5.fit(df_prep)

In [37]:
# Display frequent itemsets.
model5.freqItemsets.show()

+--------------+----+
|         items|freq|
+--------------+----+
|       [16017]|   1|
|       [19907]|  42|
| [19907, 8113]|   1|
|[19907, 19743]|   1|
|       [15497]|   1|
|       [12694]|   2|
|       [20648]|   1|
|         [661]|   2|
|        [7036]|   2|
|        [3803]|   1|
|       [17443]|   5|
|       [27285]|   1|
|       [18575]|   4|
|        [5144]|   2|
|       [12085]|   1|
|        [5979]|   1|
|       [12028]|   1|
|       [32186]|   1|
|        [2706]|   1|
|        [2035]|   1|
+--------------+----+
only showing top 20 rows



In [38]:
# Display generated association rules.
model5.associationRules.show()

+--------------------+----------+------------------+------------------+
|          antecedent|consequent|        confidence|              lift|
+--------------------+----------+------------------+------------------+
|[1810, 6329, 1214...|    [4493]|               1.0|           49333.0|
|[1810, 6329, 1214...|   [13390]|               1.0|           49333.0|
|[1810, 6329, 1214...|    [2150]|               1.0|32888.666666666664|
|[1810, 6329, 1214...|    [1686]|               1.0|           49333.0|
|               [706]|   [11366]|               1.0|           98666.0|
|[2791, 28204, 19052]|    [1858]|               1.0|           98666.0|
|[2791, 28204, 19052]|    [9545]|               1.0|           49333.0|
|[2791, 28204, 19052]|   [18100]|               1.0|           49333.0|
|[2791, 28204, 19052]|   [12083]|               1.0|           98666.0|
|             [25940]|   [31413]|0.3333333333333333|32888.666666666664|
|  [1858, 2791, 9545]|   [19052]|               1.0|32888.666666

In [39]:
# Transform examines the input items against all the association rules and summarize the
# Consequents as prediction
model5.transform(df_prep).show(50, truncate=False)

+-----+--------------+----------------------------------------------------------------------------------------------+
|id   |items         |prediction                                                                                    |
+-----+--------------+----------------------------------------------------------------------------------------------+
|10096|[21884]       |[28967, 1307, 18103]                                                                          |
|10351|[9274]        |[]                                                                                            |
|10436|[15473]       |[]                                                                                            |
|1090 |[8965]        |[12118, 26832, 21009]                                                                         |
|11078|[22549]       |[9796]                                                                                        |
|11332|[12206]       |[]                                

### 4.3 Example of I/O Product and Recomendation
This Brazilian ecommerce public dataset of orders made at Olist Store doesn't has any product name data, so I just give the product ID as a result of recommendation output.

In [40]:
# Make a new dataframe as an input
df_input = spark.createDataFrame([
                ('0',['8614']),
                ('1',['25668']),
                ('2',['19743']),
                ('3',['2791','28204','19052']),
                ('4',['20538','25213'])
            ],['id','items'])

I'll use two models to get the prediction as recommendation ouput and let's see the difference

In [41]:
# 1. Get the prediction from Model 4 (minSupport=0.00005, minConfidence=0.00005)
model4.transform(df_input).show(truncate=False)

+---+--------------------+-------------------+
|id |items               |prediction         |
+---+--------------------+-------------------+
|0  |[8614]              |[7365, 10841, 1518]|
|1  |[25668]             |[]                 |
|2  |[19743]             |[6980, 31285]      |
|3  |[2791, 28204, 19052]|[]                 |
|4  |[20538, 25213]      |[]                 |
+---+--------------------+-------------------+



In [42]:
# 2. Get the prediction from Model 5 (minSupport=0.000005, minConfidence=0.000005)
model5.transform(df_input).show(truncate=False)

+---+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |items               |prediction                                                                                                                                            |
+---+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|0  |[8614]              |[7365, 7080, 10841, 29001, 30305, 10152, 1518, 22670, 23979, 27248, 9879, 17622, 21370, 31467]                                                        |
|1  |[25668]             |[18836]                                                                                                                                               |
|2  |[19743]             |[19907, 7573, 15938, 6980, 5441, 658, 30741, 30950, 7651, 31270, 10339, 1291, 32645,

## 5. Conclusion
Based on the test results above, we can conclude that the higher minimum support and minimum confidence of a model, the more accurate the prediction results are.

## 6. References

* https://spark.apache.org/docs/2.3.0/ml-frequent-pattern-mining.html
* https://www.qubole.com/resources/pyspark-cheatsheet/
* https://stackoverflow.com/